In [7]:
import math
import time
import random
import statistics

#Option payoff struck at K
S0, T, V, R, D = 100, 0.5, 0.5, 0, 0
step=0.01
def option_price(flag,K,S0,T,V,R,D):
    random.seed(10)
    mu, sigma, disc = (R-D-0.5*V*V)*T, V*math.sqrt(T), math.exp(-R*T)
    stock_prices = [S0*random.lognormvariate(mu,sigma) for i in range(1000000)]
    if flag == 'c':
        def payoff(K):
            return lambda S:(S-K) if S>K else 0
    elif flag == 'p':
        def payoff(K):
            return lambda S:(K-S) if K>S else 0
    return statistics.mean(map(payoff(K),stock_prices))*disc

#Greeks struck at K
def Delta(flag,K,S0,T,V,R,D):
    return (option_price(flag,K,S0*(1+step),T,V,R,D)-option_price(flag,K,S0*(1-step),T,V,R,D))/(2*step*S0)

def Gamma(flag,K,S0,T,V,R,D):
    return (option_price(flag,K,S0*(1+step),T,V,R,D)-2*option_price(flag,K,S0,T,V,R,D)+option_price(flag,K,S0*(1-step),T,V,R,D))/((step*S0)*(step*S0))

def Vega(flag,K,S0,T,V,R,D):
    return (option_price(flag,K,S0,T,V*(1+step),R,D)-option_price(flag,K,S0,T,V*(1-step),R,D))/(2*step*V)

#At the money call    
print('At the money call:',option_price('c',100,100,T,V,R,D),'\n\tDelta:',Delta('c',100,100,T,V,R,D),'\n\tGamma:',Gamma('c',100,100,T,V,R,D),'\n\tVega:',Vega('c',100,100,T,V,R,D))

#At the money put
print('At the money put:',option_price('p',100,100,T,V,R,D),'\n\tDelta:',Delta('p',100,100,T,V,R,D),'\n\tGamma:',Gamma('p',100,100,T,V,R,D),'\n\tVega:',Vega('p',100,100,T,V,R,D))

#call struck at 110
print('Call struck at 110:',option_price('c',110,100,T,V,R,D),'\n\tDelta:',Delta('c',110,100,T,V,R,D),'\n\tGamma:',Gamma('c',110,100,T,V,R,D),'\n\tVega:',Vega('c',110,100,T,V,R,D))

#put struck at 90
print('Put struck at 90:',option_price('p',90,100,T,V,R,D),'\n\tDelta:',Delta('p',90,100,T,V,R,D),'\n\tGamma:',Gamma('p',90,100,T,V,R,D),'\n\tVega:',Vega('p',90,100,T,V,R,D))

At the money call: 13.984884106748042 
	Delta: 0.5691283264579567 
	Gamma: 0.010999176469290717 
	Vega: 27.659964967589445
At the money put: 14.049239239278938 
	Delta: -0.430228122216735 
	Gamma: 0.010999176469292493 
	Vega: 27.800906266580583
Call struck at 110: 10.21741067551713 
	Delta: 0.46193926061603463 
	Gamma: 0.01130604804680857 
	Vega: 27.97444559471085
Put struck at 90: 8.923934773735926 
	Delta: -0.317666499518352 
	Gamma: 0.010130201050449372 
	Vega: 25.224853616082044


In [8]:
from scipy.stats import norm
from math import *
def BlackScholes(CallPutFlag,S,K,T,r,d,v):
    d1 = (log(float(S)/K)+((r-d)+v*v/2.)*T)/(v*sqrt(T))
    d2 = d1-v*sqrt(T)
    if CallPutFlag=='c':
        return S*exp(-d*T)*norm.cdf(d1)-K*exp(-r*T)*norm.cdf(d2)
    if CallPutFlag=='p':
        return K*exp(-r*T)*norm.cdf(-d2)-S*exp(-d*T)*norm.cdf(-d1)
    
print('At the money call:',BlackScholes('c',100,100,0.5,0,0,0.5),'\nAt the money put:',BlackScholes('p',100,100,0.5,0,0,0.5))
print('\nCall struck at 110:',BlackScholes('c',100,110,0.5,0,0,0.5),'\nPut struck at 90:',BlackScholes('p',100,90,0.5,0,0,0.5))

At the money call: 14.031620480133384 
At the money put: 14.031620480133384

Call struck at 110: 10.259312807045426 
Put struck at 90: 8.910830186741848


In [9]:
def Black_Scholes_Greeks(CallPutFlag, S, K, T, r, d, v):
    T_sqrt = sqrt(T)
    d1 = (log(float(S)/K)+((r-d)+v*v/2.)*T)/(v*T_sqrt)
    d2 = d1-v*T_sqrt
    if CallPutFlag == 'c':
        Delta = norm.cdf(d1)
        Gamma = norm.pdf(d1)/(S*v*T_sqrt)
        Vega = S * T_sqrt*norm.pdf(d1)
        return Delta,Gamma,Vega
    if CallPutFlag == 'p':
        Delta = -norm.cdf(-d1)
        Gamma = norm.pdf(d1)/(S*v*T_sqrt)
        Vega = S * T_sqrt*norm.pdf(d1)
        return Delta, Gamma, Vega

print('At the money call:',Black_Scholes_Greeks('c', 100, 100, 0.5, 0, 0, 0.5),
'\nAt the money put:',Black_Scholes_Greeks('p', 100, 100, 0.5, 0, 0, 0.5),
'\nCall struck at 110:',Black_Scholes_Greeks('c', 100, 110, 0.5, 0, 0, 0.5),
'\nPut struck at 90',Black_Scholes_Greeks('p', 100, 90, 0.5, 0, 0, 0.5))

At the money call: (0.5701581024006669, 0.011108852695966625, 27.772131739916567) 
At the money put: (-0.4298418975993331, 0.011108852695966625, 27.772131739916567) 
Call struck at 110: (0.46303074782871545, 0.011235307769028785, 28.088269422571965) 
Put struck at 90 (-0.3174714544467202, 0.010081063063806543, 25.20265765951636)
